In [1]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, InputLayer, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

In [34]:
WINDOW_SIZE = 10
NUMBER_OF_FEATURES = 6
NUMBER_OF_CLASSES = 2
DATA_FOLDER = 'C:\Dev\\vr-kat-project-python-2\processed-training-data\\4-PROCESSED-DATA\TRAIN2\\'


ALL_X_TRAIN_CURRVEL_FILE_PATH = 'C:\\Dev\\vr-kat-project-python-2\\NeuralNetwork\\np-saved-data\\ALL-X-TRAIN-CURRVEL-V0.npy'
ALL_Y_TRAIN_CURRVEL_FILE_PATH = 'C:\\Dev\\vr-kat-project-python-2\\NeuralNetwork\\np-saved-data\\ALL-Y-TRAIN-CURRVEL-V0.npy'

ALL_X_TRAIN_ALLVEL_FILE_PATH = 'C:\\Dev\\vr-kat-project-python-2\\NeuralNetwork\\np-saved-data\\ALL-X-TRAIN-ALLVEL-V0.npy'
ALL_Y_TRAIN_ALLVEL_FILE_PATH = 'C:\\Dev\\vr-kat-project-python-2\\NeuralNetwork\\np-saved-data\\ALL-Y-TRAIN-ALLVEL-V0.npy'

ALL_X_TRAIN_CURRVEL_NORM1_FILE_PATH = 'C:\\Dev\\vr-kat-project-python-2\\NeuralNetwork\\np-saved-data\\ALL-X-TRAIN-CURRVEL-V1.npy'
ALL_Y_TRAIN_CURRVEL_NORM1_FILE_PATH = 'C:\\Dev\\vr-kat-project-python-2\\NeuralNetwork\\np-saved-data\\ALL-Y-TRAIN-CURRVEL-V1.npy'

ALL_X_TRAIN_ALLVEL_NORM1_FILE_PATH = 'C:\\Dev\\vr-kat-project-python-2\\NeuralNetwork\\np-saved-data\\ALL-X-TRAIN-ALLVEL-V1.npy'
ALL_Y_TRAIN_ALLVEL_NORM1_FILE_PATH = 'C:\\Dev\\vr-kat-project-python-2\\NeuralNetwork\\np-saved-data\\ALL-Y-TRAIN-ALLVEL-V1.npy'


In [4]:
file_names = [file for file in os.listdir(DATA_FOLDER) if file.endswith('.xlsx') and os.path.isfile(os.path.join(DATA_FOLDER, file))]

# TEMP LIMITER REMOVE WHEN READY
#file_names = [file_names[20]]

print(file_names)

['PROC-TRAIN2-SIDESTEPS-L-LAR-120BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-30BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-40BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-60BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-LAR-80BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-MED-110BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-MED-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-MED-220BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-MED-40BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-MED-55BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-L-MED-80BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-120BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-30BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-40BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-60BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-LAR-80BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-MED-110BPM.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-MED-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-SIDESTEPS-R-MED-220BPM-AUGMENT.xl

In [5]:
def normalizeSensorData(sensor_input):
    if(sensor_input < 0):
        return np.abs(sensor_input/360)
    else:
        return 0.5 + np.abs(sensor_input/360)


In [6]:
def df2Xy(df, windowSize=5, fetchAllSpeed=False):
  X, y = [], []


  for i in range( len(df) - windowSize ):
    # inputs: X rows
    # form a new input which has size of our windowSize
    input = []

    # loop through each row in our windowsize
    for j in range(windowSize):
        # fetch sensor data for this row
        row_values = df.loc[i + j, ['L_Pitch', 'L_Roll', 'R_Pitch', 'R_Roll']].values.tolist()
        
        # If j is the last element, fetch X_Vel and Z_Vel
        if (j == windowSize - 1) or (fetchAllSpeed == True):
            row_values.extend(df.loc[i + j, ['X_Vel', 'Z_Vel']].values.tolist())
        else:
            # Fill columns 5 and 6 with zeros
            row_values.extend([0.0, 0.0])

        # add row values to the input
        input.append(row_values)

    # add our input to our total inputs, marked as X
    X.append(input)


    # outputs: y labels
    label = df.loc[i + windowSize, ['X_Vel', 'Z_Vel']].values.tolist()

    y.append(label)


  return (np.array(X), np.array(y))

In [7]:
ALL_X_TRAIN_CURRVEL = np.empty((0, WINDOW_SIZE, NUMBER_OF_FEATURES))  # List to store all X training data
ALL_Y_TRAIN_CURRVEL = np.empty((0, NUMBER_OF_CLASSES))  # List to store all Y training data

ALL_X_TRAIN_ALLVEL = np.empty((0, WINDOW_SIZE, NUMBER_OF_FEATURES))  # List to store all X training data
ALL_Y_TRAIN_ALLVEL = np.empty((0, NUMBER_OF_CLASSES))  # List to store all Y training data

In [8]:
def processData(ALL_X_TRAIN_CURRVEL, ALL_Y_TRAIN_CURRVEL, ALL_X_TRAIN_ALLVEL, ALL_Y_TRAIN_ALLVEL):
    for fileName in file_names:
        # Read the Excel file
        df = pd.read_excel(DATA_FOLDER + fileName)

        # NORMALIZE THE SENSOR DATA
        df["L_Pitch"] = df["L_Pitch"].apply(normalizeSensorData)
        df["L_Roll"] = df["L_Roll"].apply(normalizeSensorData)
        df["R_Pitch"] = df["R_Pitch"].apply(normalizeSensorData)
        df["R_Roll"] = df["R_Roll"].apply(normalizeSensorData)

        xTrain_CurrVel, yTrain_CurrVel = df2Xy(df, WINDOW_SIZE, False)
        xTrain_AllVel, yTrain_AllVel = df2Xy(df, WINDOW_SIZE, True)

        ALL_X_TRAIN_CURRVEL = np.concatenate([ALL_X_TRAIN_CURRVEL, xTrain_CurrVel])
        ALL_Y_TRAIN_CURRVEL = np.concatenate([ALL_Y_TRAIN_CURRVEL, yTrain_CurrVel])

        ALL_X_TRAIN_ALLVEL = np.concatenate([ALL_X_TRAIN_ALLVEL, xTrain_AllVel])
        ALL_Y_TRAIN_ALLVEL = np.concatenate([ALL_Y_TRAIN_ALLVEL, yTrain_AllVel])

        
        print('xTrain_CurrVel.shape:', xTrain_CurrVel.shape, ' xTrain_CurrVel.shape:', xTrain_CurrVel.shape)
        print('ALL_X_TRAIN_CURRVEL.shape:', ALL_X_TRAIN_CURRVEL.shape, ' ALL_Y_TRAIN_CURRVEL.shape:', ALL_Y_TRAIN_CURRVEL.shape)
        print('-------------------------------')

    # Assuming your numpy array is called 'data_array'
    np.save(ALL_X_TRAIN_CURRVEL_FILE_PATH, ALL_X_TRAIN_CURRVEL)
    np.save(ALL_Y_TRAIN_CURRVEL_FILE_PATH, ALL_Y_TRAIN_CURRVEL)
    np.save(ALL_X_TRAIN_ALLVEL_FILE_PATH, ALL_X_TRAIN_ALLVEL)
    np.save(ALL_Y_TRAIN_ALLVEL_FILE_PATH, ALL_Y_TRAIN_ALLVEL)


In [23]:
if os.path.exists(ALL_X_TRAIN_CURRVEL_FILE_PATH):
    ALL_X_TRAIN_CURRVEL = np.load(ALL_X_TRAIN_CURRVEL_FILE_PATH)
    ALL_Y_TRAIN_CURRVEL = np.load(ALL_Y_TRAIN_CURRVEL_FILE_PATH)
    ALL_X_TRAIN_ALLVEL = np.load(ALL_X_TRAIN_ALLVEL_FILE_PATH)
    ALL_Y_TRAIN_ALLVEL = np.load(ALL_Y_TRAIN_ALLVEL_FILE_PATH)
else:
    processData(ALL_X_TRAIN_CURRVEL, ALL_Y_TRAIN_CURRVEL, ALL_X_TRAIN_ALLVEL, ALL_Y_TRAIN_ALLVEL)

print(ALL_X_TRAIN_CURRVEL.shape)
print(ALL_Y_TRAIN_CURRVEL.shape)


(229679, 10, 6)
(229679, 2)


In [29]:
# assume the range is from -8 to 8
def normalizeXVelocity(velocity_input):
    return ((velocity_input + 8)/16)

# assume thhe range from 0 to 8
def normalizeZVelocity(velocity_input):
    return np.abs(velocity_input/8)

In [35]:
# given we have non normalize velocity, lets normalize velocity and hold it in our V1 data. 
ALL_X_TRAIN_CURRVEL_CURR_NORM1 = np.copy(ALL_X_TRAIN_CURRVEL)
ALL_Y_TRAIN_CURRVEL_CURR_NORM1 = np.copy(ALL_Y_TRAIN_CURRVEL)
ALL_X_TRAIN_CURRVEL_ALLVEL_NORM1 = np.copy(ALL_X_TRAIN_ALLVEL)
ALL_Y_TRAIN_CURRVEL_ALLVEL_NORM1 = np.copy(ALL_Y_TRAIN_ALLVEL)

#loop through each data training data
for i in range(ALL_X_TRAIN_CURRVEL_CURR_NORM1.shape[0]):
    #loop through each row in our WINDOW_SIZE
    for j in range(ALL_X_TRAIN_CURRVEL_CURR_NORM1.shape[1]):

        #normalize X annd Z velocity using our function
        #index 4 of X - X Velocity
        #index 5 of X - Z Velocity
        ALL_X_TRAIN_CURRVEL_CURR_NORM1[i,j,4] = normalizeXVelocity(ALL_X_TRAIN_CURRVEL_CURR_NORM1[i,j,4])
        ALL_X_TRAIN_CURRVEL_CURR_NORM1[i,j,5] = normalizeZVelocity(ALL_X_TRAIN_CURRVEL_CURR_NORM1[i,j,5])

        #index 0 of Y - X Velocity
        #index 1 of Y - Z Velocity
        ALL_Y_TRAIN_CURRVEL_CURR_NORM1[i,0] = normalizeXVelocity(ALL_Y_TRAIN_CURRVEL_CURR_NORM1[i,0])
        ALL_Y_TRAIN_CURRVEL_CURR_NORM1[i,1] = normalizeZVelocity( ALL_Y_TRAIN_CURRVEL_CURR_NORM1[i,1])

        ALL_X_TRAIN_CURRVEL_ALLVEL_NORM1[i,j,4] = normalizeXVelocity(ALL_X_TRAIN_CURRVEL_ALLVEL_NORM1[i,j,4])
        ALL_X_TRAIN_CURRVEL_ALLVEL_NORM1[i,j,5] = normalizeZVelocity(ALL_X_TRAIN_CURRVEL_ALLVEL_NORM1[i,j,5])

        ALL_Y_TRAIN_CURRVEL_ALLVEL_NORM1[i,0] = normalizeXVelocity(ALL_Y_TRAIN_CURRVEL_ALLVEL_NORM1[i,0])
        ALL_Y_TRAIN_CURRVEL_ALLVEL_NORM1[i,1] = normalizeZVelocity( ALL_Y_TRAIN_CURRVEL_ALLVEL_NORM1[i,1])


# Assuming your numpy array is called 'data_array'
np.save(ALL_X_TRAIN_CURRVEL_NORM1_FILE_PATH, ALL_X_TRAIN_CURRVEL_CURR_NORM1)
np.save(ALL_Y_TRAIN_CURRVEL_NORM1_FILE_PATH, ALL_Y_TRAIN_CURRVEL_CURR_NORM1)
np.save(ALL_X_TRAIN_ALLVEL_NORM1_FILE_PATH, ALL_X_TRAIN_CURRVEL_ALLVEL_NORM1)
np.save(ALL_Y_TRAIN_ALLVEL_NORM1_FILE_PATH, ALL_Y_TRAIN_CURRVEL_ALLVEL_NORM1)
